In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import import_ipynb
from CandidateExtraction import *
import os
from keras.models import load_model
import json
from Tesseract import *
from keras.models import load_model
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

importing Jupyter notebook from CandidateExtraction.ipynb
importing Jupyter notebook from Tesseract.ipynb


2023-07-19 12:13:37.669980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")


In [3]:
def getNeighbourInfo(candidates, data):
    file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/wordEmbeddings.json")
    dict = json.load(file)

    df = candidates[1]
    
    for col in dict.keys():
        df[col] = 0
        df[col+"_X"] = 0
        df[col+"_Y"] = 0
    
    for index in df.index:
        Neighbours = []
        X = df["x1"][index] + df["x2"][index]
        Y = df["y1"][index] + df["y2"][index]
        X/=2
        Y/=2
        for idx in data.index:
            text = data["text"][idx].lower()
            X2 = data["X"][idx]
            Y2 = data["Y"][idx]
            if Y-Y2 >= 0 and Y-Y2 <= 0.1  and X2<X and text in dict.keys():
                df[text][index] = dict[text]
                df[text+"_X"][index] = X-X2
                df[text+"_Y"][index] = Y-Y2

In [4]:
def createCandidates(file):

    data, candidateInfo = get_candidates(file)

    data = data[["text", "X", "Y"]]


    getNeighbourInfo(candidateInfo, data)

    return candidateInfo[1]

    # result = pd.concat(candidateInfo)

    # result.to_csv("Candidates/" + file.strip("/")[1])
    



In [6]:
df = createCandidates("Images/10.jpeg")
df

,text,x1,y1,x2,y2,Class,invoice,invoice_X,invoice_Y,pitstop,...,tunga_Y,chambers,chambers_X,chambers_Y,kh,kh_X,kh_Y,kkbkinb,kkbkinb_X,kkbkinb_Y
0,31-Dec-22,0.750742,0.184506,0.826901,0.193357,invoice_date,-0.038794,0.157158,0.001049,0,...,0,0,0,0,0,0,0,0,0,0
1,+91,0.329160,0.627065,0.359365,0.636737,invoice_date,0.000000,0.000000,0.000000,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
imagesDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
for file in imagesDir:
    print(file)
    df = createCandidates(file)
    df.head()
    os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")
    df.to_csv(file + ".csv")
    os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")

20.jpeg
23.jpeg
68.jpeg
106.jpeg
55.jpeg
28.jpeg
101.jpeg
79.jpeg
18.jpeg
82.jpeg
39.jpeg
64.jpeg
94.jpeg
22.jpeg
75.jpeg
27.jpeg
4.jpeg
104.jpeg
17.jpeg
5.jpeg
9.jpeg
78.jpeg
3.jpeg
24.jpeg
25.jpeg
34.jpeg
54.jpeg
91.jpeg
7.jpeg
1.jpeg
41.jpeg
57.jpeg
33.jpeg
21.jpeg
85.jpeg
100.jpeg
14.jpeg
81.jpeg
38.jpeg
11.jpeg
6.jpeg
30.jpeg
43.jpeg
80.jpeg
71.jpeg
76.jpeg
37.jpeg
47.jpeg
96.jpeg
15.jpeg
83.jpeg
29.jpeg
36.jpeg
84.jpeg
58.jpeg
12.jpeg
53.jpeg
35.jpeg
107.jpeg
10.jpeg
67.jpeg
110.jpeg
97.jpeg
32.jpeg
108.jpeg
60.jpeg
8.jpeg
61.jpeg
88.jpeg
40.jpeg
2.jpeg
16.jpeg
90.jpeg
89.jpeg
109.jpeg
56.jpeg
49.jpeg
87.jpeg
50.jpeg
48.jpeg
63.jpeg
42.jpeg
65.jpeg
74.jpeg
45.jpeg
66.jpeg
44.jpeg
52.jpeg
13.jpeg
103.jpeg
105.jpeg
59.jpeg
31.jpeg
99.jpeg
77.jpeg
73.jpeg
72.jpeg
98.jpeg
95.jpeg
19.jpeg
70.jpeg
51.jpeg
26.jpeg
92.jpeg
102.jpeg
69.jpeg
93.jpeg
86.jpeg
46.jpeg
62.jpeg
